In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import matplotlib.pylab as plt
import numpy as np
import pickle, os, warnings, sys, random, logging, librosa, json, nemo
import soundfile as sf
from tqdm.auto import tqdm
from ruamel.yaml import YAML
import nemo.collections.asr as nemo_asr
from omegaconf import DictConfig
import pytorch_lightning as pl
from Cfg import Cfg
from RecordingCorpus import RecordingCorpus
from multiprocessing import Pool
from contextlib import closing

warnings.filterwarnings("ignore")
logger = logging.getLogger()
logger.setLevel(logging.INFO)

[NeMo W 2020-10-14 14:18:44 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text.AudioToCharDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2020-10-14 14:18:44 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text.AudioToBPEDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2020-10-14 14:18:44 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text.AudioLabelDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2020-10-14 14:18:44 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text._TarredAudioToTextDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2020-10-14 14:18:44 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text.TarredAudioToCharDataset'> is experimental,

## Fix up rates to 16K

In [5]:
C = Cfg('NIST', 16000, 'amharic') 
audio_split_dir=f'{C.build_dir}/audio_split'

In [6]:
!ls {audio_split_dir}

BABEL_OP3_307_14229_20140503_233516_inLine_1012440_1039240.wav
BABEL_OP3_307_14229_20140503_233516_inLine_1073800_1101880.wav
BABEL_OP3_307_14229_20140503_233516_inLine_1128680_1145880.wav
BABEL_OP3_307_14229_20140503_233516_inLine_1166680_1185240.wav
BABEL_OP3_307_14229_20140503_233516_inLine_1201720_1215880.wav
BABEL_OP3_307_14229_20140503_233516_inLine_1248920_1272040.wav
BABEL_OP3_307_14229_20140503_233516_inLine_1302360_1318360.wav
BABEL_OP3_307_14229_20140503_233516_inLine_1357480_1363640.wav
BABEL_OP3_307_14229_20140503_233516_inLine_1387480_1393160.wav
BABEL_OP3_307_14229_20140503_233516_inLine_1446440_1468120.wav
BABEL_OP3_307_14229_20140503_233516_inLine_1507560_1542680.wav
BABEL_OP3_307_14229_20140503_233516_inLine_1605080_1621640.wav
BABEL_OP3_307_14229_20140503_233516_inLine_1656600_1706280.wav
BABEL_OP3_307_14229_20140503_233516_inLine_1779160_1800280.wav
BABEL_OP3_307_14229_20140503_233516_inLine_1835320_1839560.wav
BABEL_OP3_307_14229_20140503_233516_inLine_186440_28028

BABEL_OP3_307_87074_20140429_185857_outLine_248200_268120.wav
BABEL_OP3_307_87074_20140429_185857_outLine_2495800_2529880.wav
BABEL_OP3_307_87074_20140429_185857_outLine_2559080_2593560.wav
BABEL_OP3_307_87074_20140429_185857_outLine_2615720_2711320.wav
BABEL_OP3_307_87074_20140429_185857_outLine_2737000_2810120.wav
BABEL_OP3_307_87074_20140429_185857_outLine_2899400_2932520.wav
BABEL_OP3_307_87074_20140429_185857_outLine_290280_318760.wav
BABEL_OP3_307_87074_20140429_185857_outLine_2939000_3031880.wav
BABEL_OP3_307_87074_20140429_185857_outLine_3049640_3084840.wav
BABEL_OP3_307_87074_20140429_185857_outLine_3117320_3143320.wav
BABEL_OP3_307_87074_20140429_185857_outLine_3317800_3328280.wav
BABEL_OP3_307_87074_20140429_185857_outLine_3358280_3377160.wav
BABEL_OP3_307_87074_20140429_185857_outLine_3405400_3478200.wav
BABEL_OP3_307_87074_20140429_185857_outLine_343560_383720.wav
BABEL_OP3_307_87074_20140429_185857_outLine_3512680_3557400.wav
BABEL_OP3_307_87074_20140429_185857_outLine_35

## Train Phase 1: Start from scratch

In [ ]:
C = Cfg('NIST', 8000, 'amharic') 
audio_split_dir=f'{C.build_dir}/audio_split'

!mv NIST/openasr20_amharic/build/audio_split NIST/openasr20_amharic/build/audio_split_8000

!mkdir -p {audio_split_dir}

from glob import glob

files=glob('NIST/openasr20_amharic/build/audio_split_8000/*.wav')

from tqdm.auto import tqdm

import soundfile as sf
import librosa

!ls NIST/openasr20_amharic/build/audo_split

for fn in tqdm(files):
    newfn = fn.replace('audio_split_8000', 'audio_split')
    y3, sr3 = librosa.load(fn,sr=16000)
    sf.write(newfn, y3, 16000)

## Train step 1: Bootstrap from pretrained model

## Train Phase 2: K-fold validation more or less

In [ ]:
from reshuffle_samples import reshuffle_samples
import nemo.collections.asr as nemo_asr
from load_recent import load_recent
from ruamel.yaml import YAML
import pytorch_lightning as pl
from Cfg import Cfg
from omegaconf import DictConfig
import os, datetime

C = Cfg('NIST', 16000, 'amharic') 

model_save_dir='save/nemo_amharic'

config_path = 'quartznet_15x5_amharic.yaml'
yaml = YAML(typ='safe')
with open(config_path) as f:
    params = yaml.load(f)
params['model']['train_ds']['manifest_filepath'] = f'{C.build_dir}/train_manifest.json'
params['model']['validation_ds']['manifest_filepath'] = f'{C.build_dir}/test_manifest.json'
params['model']['optim']['lr'] = 0.001

class ModelCheckpointAtEpochEnd(pl.callbacks.ModelCheckpoint):
    def on_epoch_end(self, trainer, pl_module):
        metrics = trainer.callback_metrics
        metrics['epoch'] = trainer.current_epoch
        trainer.checkpoint_callback.on_validation_end(trainer, pl_module)

pid=os.getpid()
dt=datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

checkpoint_callback = ModelCheckpointAtEpochEnd(
    filepath=model_save_dir+'/amharic_'+f'{dt}_{pid}'+'_{epoch:02d}',
    verbose=True,
    save_top_k=-1,
    save_weights_only=False,
    period=1)

trainer = pl.Trainer(gpus=[0], max_epochs=20, amp_level='O1', precision=16, checkpoint_callback=checkpoint_callback)

while True:
    reshuffle_samples(C)
    model = nemo_asr.models.EncDecCTCModel(cfg=DictConfig(params['model']), trainer=trainer)
    load_recent(C, model)
    trainer.fit(model)